## 모듈 임포트 및 데이터 로딩

In [1]:
import numpy as np
import pandas as pd

from dash import Dash
import dash_html_components as html
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import dash_table
from dash.dependencies import Input,Output,State
import plotly.express as px
import plotly.graph_objects as go
import dash

In [2]:
# 한글 폰트 사용을 위해서 세팅
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/gulim.ttc"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

In [3]:
df = pd.read_csv('C:/Users/admin/Desktop/mini2/data/df_seoul_final.csv')

In [4]:
df

,년월,카드이용지역,업종명,거주지역_시도,거주지역_시군구,연령대,성별,가구생애주기,이용고객수_명,이용금액_원,이용건수_건,날짜
0,201901,강남구,건강보조식품 소매업,강원,강릉시,60s,F,노년가구,7,1181300,8,2019-01-01
1,201901,강남구,건강보조식품 소매업,강원,속초시,50s,F,성인자녀가구,3,756000,4,2019-01-01
2,201901,강남구,건강보조식품 소매업,강원,원주시,50s,F,성인자녀가구,6,2480710,14,2019-01-01
3,201901,강남구,건강보조식품 소매업,강원,원주시,60s,F,노년가구,6,3604450,7,2019-01-01
4,201901,강남구,건강보조식품 소매업,강원,춘천시,50s,F,성인자녀가구,4,1952750,11,2019-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...
6259497,202003,중랑구,화장품 및 방향제 소매업,인천,서구,40s,F,중고생자녀가구,3,25800,3,2020-03-01
6259498,202003,중랑구,화장품 및 방향제 소매업,인천,서구,50s,M,성인자녀가구,9,231700,10,2020-03-01
6259499,202003,중랑구,화장품 및 방향제 소매업,인천,서구,50s,F,성인자녀가구,16,288700,17,2020-03-01
6259500,202003,중랑구,화장품 및 방향제 소매업,인천,서구,60s,M,노년가구,6,69600,6,2020-03-01


In [5]:
pred = pd.read_csv('C:/Users/admin/Desktop/mini2/data/predict.csv')

In [6]:
pred

,날짜,카드이용지역,업종명,이용금액_원
0,202108,강남구,건강보조식품 소매업,6.928585e+07
1,202108,강남구,골프장 운영업,1.616655e+08
2,202108,강남구,과실 및 채소 소매업,2.247819e+08
3,202108,강남구,관광 민예품 및 선물용품 소매업,1.116968e+07
4,202108,강남구,그외 기타 스포츠시설 운영업,1.357063e+07
...,...,...,...,...
2920,202110,중랑구,택시 운송업,2.045791e+08
2921,202110,중랑구,피자 햄버거 샌드위치 및 유사 음식점업,4.449446e+08
2922,202110,중랑구,한식 음식점업,4.986959e+09
2923,202110,중랑구,호텔업,5.884720e+07


## Front-End

In [7]:
dropdown = dcc.Dropdown(
    id = 'kind',
    options = [{'label': i, 'value': i} for i in sorted(df.업종명.unique())],
    style = {'color':'black'}
)
bar_graph = dcc.Graph(
    id = 'bar-graph'
)
bar_graph2 = dcc.Graph(
    id = 'bar-graph2'
)
bar_graph3 = dcc.Graph(
    id = 'bar-graph3'
)
line_graph = dcc.Graph(
    id = 'line-graph'
)
dropdown2 = dcc.Dropdown(
    id = 'card',
    options= [{'label': i, 'value': i} for i in sorted(df.카드이용지역.unique())],
    style = {'color':'black'}
)
radioitem = dcc.RadioItems(
    id = 'month',
    options = [
        {'label':'8월','value':202108},
        {'label':'9월','value':202109},
        {'label':'10월','value':202110}
    ],
    style = {'color':'black', "textAlign": "center"}
)
bar_graph4 = dcc.Graph(
    id = 'bar-graph4'
)

In [8]:
controls = dbc.Card(
    [
        dbc.FormGroup(
            [
                dbc.Label("업종"),
                dropdown
            ]
        )
    ],
    body = True
)
controls2 = dbc.Card(
    [
        dbc.FormGroup(
            [
                line_graph
            ]
        ),
        dbc.FormGroup(
            [
                radioitem,
                bar_graph4
            ]
        )
    ]
)
card_content = [
    dbc.CardHeader("월예상 매출"),
    dbc.CardBody(
        [
            html.H4(id = 'predict', className="card-title", style={"textAlign": "center"}),
            html.H4(id = 'average', className="card-title", style={"textAlign": "center"}),
            html.P("월예상 매출이 전체 지역 평균 월매출보다 높을 경우" , className="card-text"),
            html.P("선택 지역의 매출이 안정적일 것입니다" , className="card-text")
        ]
    ),
]
controls3= dbc.Card(
    [
        dbc.FormGroup(
            [
                dbc.Label("지역"),
                dropdown2
            ]
        ),
        dbc.FormGroup(
            [
                dbc.Label("지역별 상세차트"),
                html.Div(id="table")
            ]
        ),
        dbc.Card(card_content, color="primary", inverse=True)
    ],
    body = True
)
card_content2 = [
    dbc.CardHeader("추천 지역"),
    dbc.CardBody(
        [
            html.H4(id = 'loc-summary', className="card-title", style={"textAlign": "center"})
        ]
    ),
]
card_content3 = [
    dbc.CardHeader("주고객 연령"),
    dbc.CardBody(
        [
            html.H4(id = 'age-summary', className="card-title", style={"textAlign": "center"})
        ]
    ),
]
card_content4 = [
    dbc.CardHeader("주고객 성별"),
    dbc.CardBody(
        [
            html.H4(id = 'sex-summary', className="card-title", style={"textAlign": "center"})
        ]
    ),
]

In [9]:
app = dash.Dash(__name__,external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = dbc.Container(
    [
        html.Br(),
        html.H4('서울시 업종별 매출 분석 및 예상 창업 컨설팅', style={"textAlign": "center"}),
        html.Hr(),
        dbc.Col(controls),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col(dbc.Card(card_content2, color="warning", inverse=True), md=6),
                dbc.Col(dbc.Card(card_content3, color="info", inverse=True), md=3),
                dbc.Col(dbc.Card(card_content4, color="success", inverse=True), md=3)
            ],
            className="mb-4",
        ),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col(bar_graph3, md=6),
                dbc.Col(bar_graph, md=3),
                dbc.Col(bar_graph2, md=3)
            ]
        
        ),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col(controls2, md=8),
                dbc.Col(controls3, md=4)
            ]
        )
    ],
    fluid=True,
    style = {'background-color':'white'}
)

## Back-End

In [10]:
# Summary card
@app.callback(
    Output('age-summary','children'),
    Input('kind','value')
)
def age_summary(kind):
    df_class_age = df.groupby(['업종명','연령대'], as_index = False).sum()
    data = df_class_age[df_class_age['업종명'] == kind].groupby('연령대').sum().reset_index()[['연령대','이용건수_건','이용금액_원']]
    data_2 = data.sort_values(by=['이용금액_원'], axis=0, ascending=False)
    return "{}".format(data_2.iloc[0][0])

@app.callback(
    Output('sex-summary','children'),
    Input('kind','value')
)
def sex_summary(kind):
    df_class_sex = df.groupby(['업종명','성별'], as_index = False).sum()
    data2 = df_class_sex[df_class_sex['업종명'] == kind].groupby('성별').sum().reset_index()[['성별','이용건수_건','이용금액_원']]
    data2_2 = data2.sort_values(by=['이용금액_원'], axis=0, ascending=False)
    return "{}".format(data2_2.iloc[0][0])

@app.callback(
    Output('loc-summary','children'),
    Input('kind','value')
)
def loc_summary(kind):
    df_class_loc = df.groupby(['업종명','카드이용지역'], as_index= False).sum()
    data3 = df_class_loc[df_class_loc['업종명']== kind].groupby('카드이용지역').sum().reset_index()[['카드이용지역','이용건수_건','이용금액_원']] 
    data3_2 = data3.sort_values(by=['이용금액_원'], axis=0, ascending=False)
    return "1위 {}   2위 {}   3위 {}".format(data3_2.iloc[0][0],data3_2.iloc[1][0],data3_2.iloc[2][0])

# Specific graph
@app.callback(
    Output('bar-graph','figure'),
    Input('kind','value')
)
def age_bar_graph(kind):
    df_class_age = df.groupby(['업종명','연령대'], as_index = False).sum()
    data = df_class_age[df_class_age['업종명'] == kind].groupby('연령대').sum().reset_index()[['연령대','이용건수_건','이용금액_원']]
    fig = px.bar(data, x = '연령대', y = '이용금액_원')
    colors = ['indianred', 'coral', 'yellow', 'yellowgreen', 'blueviolet'] * 2
    fig.update_traces(marker_color = colors)
    fig.update_layout(title_text='연령별 총 이용금액 (원)', margin=dict(l=5, r=5, t=30, b=5))
    #fig.update_layout(margin=dict(l=10, r=10, t=10, b=10))
    return fig

@app.callback(
    Output('bar-graph2','figure'),
    Input('kind','value')
)
def sex_bar_graph(kind):
    df_class_sex = df.groupby(['업종명','성별'], as_index = False).sum()
    data2 = df_class_sex[df_class_sex['업종명'] == kind].groupby('성별').sum().reset_index()[['성별','이용건수_건','이용금액_원']]
    fig2 = px.bar(data2, x = '성별', y = '이용금액_원')
    colors = ['yellowgreen', 'yellow']
    fig2.update_traces(marker_color = colors)
    fig2.update_layout(title_text='성별 총 이용금액 (원)', margin=dict(l=5, r=5, t=30, b=5))
    return fig2

@app.callback(
    Output('bar-graph3','figure'),
    Input('kind','value')
)
def loc_bar_graph(kind):
    df_class_loc = df.groupby(['업종명','카드이용지역'], as_index= False).sum()
    data3 = df_class_loc[df_class_loc['업종명']== kind].groupby('카드이용지역').sum().reset_index()[['카드이용지역','이용건수_건','이용금액_원']] 
    fig3 = px.bar(data3, x = '카드이용지역', y = '이용금액_원', text = data3.이용금액_원, color_discrete_sequence=px.colors.sequential.RdBu)
    fig3.update_xaxes(tickangle=45, tickfont=dict(family='Rockwell'))
    fig3.update_traces(texttemplate='%{text:.2s}', textposition='outside')
    colors = ['indianred', 'coral', 'yellow', 'yellowgreen', 'blueviolet'] * 10
    fig3.update_traces(marker_color = colors)
    fig3.update_layout(title_text='지역별 총 이용금액 (원)', margin=dict(l=5, r=5, t=30, b=5))
    return fig3

@app.callback(
    Output('line-graph','figure'),
    Input('kind','value')
)
def date_line_graph(kind):
    df_class_date = df.groupby(['업종명','날짜','카드이용지역'], as_index = False).sum()
    data4 = df_class_date[df_class_date['업종명'] == kind].groupby(['날짜','카드이용지역']).sum().reset_index()[['날짜','카드이용지역','이용건수_건','이용금액_원']] 
    fig4 = px.line(data4, x="날짜", y="이용금액_원", color='카드이용지역') #palette='Set2', hue='카드이용지역'
    fig4.update_layout(title_text='날짜별 총 이용금액 (원)', height=400)
    return fig4

@app.callback(
    Output('table','children'),
    [Input('kind','value'),
    Input('card','value')]
)
def date_table(kind, card):
    df_class_date = df.groupby(['업종명','년월','카드이용지역'], as_index = False).sum()
    data4 = df_class_date[df_class_date['업종명']== kind].groupby(['년월','카드이용지역']).sum().reset_index()[['년월','카드이용지역','이용건수_건','이용금액_원']]
    data4_2 = data4[data4['카드이용지역'] == card].reset_index()[['년월','이용건수_건','이용금액_원']]
    return html.Div(
        [
            dash_table.DataTable(
                data=data4_2.to_dict("rows"),
                columns=[{"id": x, "name": x} for x in data4_2.columns],
            )
        ]
    )

@app.callback(
    Output('bar-graph4','figure'),
    [Input('kind','value'),
    Input('month','value')]
)
def loc_bar_graph(kind,month):
    data5 = pred[(pred['업종명'] == kind)&(pred['날짜'] == month)]
    fig5 = px.bar(data5, x = '카드이용지역', y = '이용금액_원', text = data5.이용금액_원, color_discrete_sequence=px.colors.sequential.RdBu)
    fig5.update_xaxes(tickangle=45, tickfont=dict(family='Rockwell'))
    fig5.update_traces(texttemplate='%{text:.2s}', textposition='outside')
    colors = ['indianred', 'coral', 'yellow', 'yellowgreen', 'blueviolet'] * 10
    fig5.update_traces(marker_color = colors)
    fig5.update_layout(title_text='지역별 예측 이용금액 (원)', height=400)
    return fig5

# final summary
@app.callback(
    Output('predict','children'),
    [Input('kind','value'),
    Input('card','value')]
)
def predict_card(kind,card):
    data6 = pred[(pred['업종명'] == kind) & (pred['카드이용지역'] == card)]
    return "선택지역 월예상 매출 : {}원".format(round(sum(data6['이용금액_원'])/3))

@app.callback(
    Output('average','children'),
    Input('kind','value')
)
def average_card(kind):
    data7 = df[df['업종명']==kind].groupby(['카드이용지역','년월'], as_index = False).sum()
    return "전체지역 월평균 매출 : {}원".format(round(sum(data7['이용금액_원'])/len(data7['이용금액_원'])))

## 서버 연결

In [11]:
app.run_server(host='192.168.45.180',port='8888')

Dash is running on http://192.168.45.180:8888/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://192.168.45.180:8888/ (Press CTRL+C to quit)
192.168.45.180 - - [28/Jul/2021 11:30:09] "GET / HTTP/1.1" 200 -
192.168.45.180 - - [28/Jul/2021 11:30:10] "GET /_dash-layout HTTP/1.1" 200 -
192.168.45.180 - - [28/Jul/2021 11:30:10] "GET /_dash-dependencies HTTP/1.1" 200 -
192.168.45.180 - - [28/Jul/2021 11:30:10] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\admin\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\admin\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\admin\Anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\admin\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\admin\Anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\admin\Anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\admin\Anaconda3\lib\site-packages\dash\dash.py", line 1078, in dispatch
    response.set_d

192.168.45.180 - - [28/Jul/2021 11:30:11] "POST /_dash-update-component HTTP/1.1" 500 -
C:\Users\admin\Anaconda3\lib\site-packages\pandas\core\frame.py:1554: FutureWarning:

Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.

192.168.45.180 - - [28/Jul/2021 11:30:14] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\admin\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\admin\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\admin\Anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\admin\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\admin\Anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\admin\Anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\admin\Anaconda3\lib\site-packages\dash\dash.py", line 1078, in dispatch
    response.set_d

...

192.168.45.180 - - [28/Jul/2021 11:30:16] "POST /_dash-update-component HTTP/1.1" 500 -
192.168.45.180 - - [28/Jul/2021 11:30:16] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.45.180 - - [28/Jul/2021 11:30:16] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.45.180 - - [28/Jul/2021 11:30:17] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.45.180 - - [28/Jul/2021 11:30:17] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.45.180 - - [28/Jul/2021 11:30:17] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.45.180 - - [28/Jul/2021 11:30:17] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.45.180 - - [28/Jul/2021 11:30:17] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\admin\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\admin\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\admin\Anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\admin\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\admin\Anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\admin\Anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\admin\Anaconda3\lib\site-packages\dash\dash.py", line 1078, in dispatch
    response.set_d

192.168.45.180 - - [28/Jul/2021 11:30:19] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\admin\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\admin\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\admin\Anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\admin\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\admin\Anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\admin\Anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\admin\Anaconda3\lib\site-packages\dash\dash.py", line 1078, in dispatch
    response.set_d

192.168.45.180 - - [28/Jul/2021 11:30:20] "POST /_dash-update-component HTTP/1.1" 500 -
192.168.45.180 - - [28/Jul/2021 11:30:22] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.45.180 - - [28/Jul/2021 11:30:23] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.45.180 - - [28/Jul/2021 11:30:23] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.45.180 - - [28/Jul/2021 11:30:24] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.45.180 - - [28/Jul/2021 11:30:24] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.45.180 - - [28/Jul/2021 11:30:25] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.45.180 - - [28/Jul/2021 11:30:25] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.45.180 - - [28/Jul/2021 11:30:26] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.45.180 - - [28/Jul/2021 11:30:26] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.45.180 - - [28/Jul/2021 11:30:26] "POST /_dash-update-component HTTP/1.1" 200 -
